# DATA 304 – Module 3, Session 2 DEMO
Encodings, malformed CSV rows, debugging workflow, and Excel pitfalls.
_Data folder: `session2_data/`_

In [ ]:
from pathlib import Path
import pandas as pd
DATA_DIR = Path("data")
list(DATA_DIR.iterdir())

## 1. Character Encodings

In [ ]:
utf8_path = DATA_DIR / "utf8_file.csv"
try:
    df_utf8 = pd.read_csv(utf8_path, encoding="ascii")
    print("No errors!")
except Exception as e:
    print("Error:", e)

In [ ]:
df_utf8 = pd.read_csv(utf8_path, encoding="utf-8")
df_utf8

In [ ]:
latin1_path = DATA_DIR / "latin1_file.csv"
try:
    df_latin1 = pd.read_csv(latin1_path)
    print("No errors!")
except UnicodeDecodeError as e:
    print("Error:", e)

In [ ]:
df_latin1 = pd.read_csv(latin1_path, encoding="latin1")
df_latin1

## 2. Malformed CSV Rows

In [ ]:
broken_path = DATA_DIR / "broken_rows.csv"
try:
    df_broken = pd.read_csv(broken_path)
    print("No errors!")
except Exception as e:
    print("Error:", e)

In [ ]:
! head data/broken_rows.csv

In [ ]:
! awk -F"," '{print "Column count: " NF}' data/broken_rows.csv | sort | uniq -c

In [ ]:
lines = open(broken_path).read().splitlines()
[(line, line.count(',')+1) for _, line in enumerate(lines)]

In [ ]:
from collections import Counter
Counter([line.count(',')+1 for line in open(broken_path)])

In [ ]:
# on_bad_lines{{‘error’, ‘warn’, ‘skip’}}, default ‘error’
df_fixed = pd.read_csv(broken_path, on_bad_lines="skip")
df_fixed

## 3. Excel Pitfalls

In [ ]:
xls_path = DATA_DIR / "messy_excel.xlsx"
xe = pd.ExcelFile(xls_path)
xe.sheet_names

In [ ]:
df_quarterly = xe.parse("Quarterly")
df_quarterly

In [ ]:
df_quarterly = xe.parse("Quarterly", header=1)
df_quarterly

In [ ]:
# drop fully empty rows
df_quarterly = df_quarterly.dropna(how="all")
df_quarterly

In [ ]:
# drop the trailing "Report total" row (match anywhere in the row)
mask_footer = df_quarterly.astype(str).apply(
    lambda r: r.str.strip().str.lower().eq("report total").any(), axis=1
)
df_quarterly = df_quarterly[~mask_footer].copy()
df_quarterly

In [ ]:
for col in df_quarterly.columns[:3]:
    df_quarterly[col] = pd.to_numeric(df_quarterly[col], errors="coerce")

df_quarterly

In [ ]:
# bad header names that should be a single column
bad_cols = list(df_quarterly.columns[3:])

# align those names to rows (truncate/exact match in length)
cities = bad_cols[:len(df_quarterly)]

# attach as a proper column
df_quarterly.loc[:, "city"] = pd.Series(cities, index=df_quarterly.index)

# drop the bogus all-NaN columns
df_quarterly = df_quarterly.drop(columns=bad_cols)

df_quarterly
 

In [ ]:
df_quarterly["city"] = (
    df_quarterly["city"]
      .apply(lambda x: x.encode("latin1", errors="ignore").decode("utf-8", errors="ignore"))
      .str.strip()
)
df_quarterly